<a href="https://colab.research.google.com/github/Remonah-3/Github_Assignment/blob/master/SimpleConv1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class SimpleConv1d:
    def __init__(self, filter_size, learning_rate=0.01):
        self.filter_size = filter_size
        self.learning_rate = learning_rate
        # Xavier initialization (float)
        self.w = np.random.randn(filter_size).astype(float) * np.sqrt(1.0 / filter_size)
        self.b = 0.0

    def forward(self, x):
        self.x = x.astype(float)
        output_length = len(self.x) - self.filter_size + 1
        self.a = np.empty(output_length, dtype=float)
        for i in range(output_length):
            self.a[i] = np.sum(self.x[i:i+self.filter_size] * self.w) + self.b
        return self.a

    def backward(self, delta_a):
        delta_w = np.zeros_like(self.w, dtype=float)
        delta_b = np.sum(delta_a)
        delta_x = np.zeros_like(self.x, dtype=float)


In [2]:
def conv1d_output_size(input_size, filter_size, padding=0, stride=1):
    return (input_size + 2*padding - filter_size)//stride + 1

print(conv1d_output_size(4, 3))


2


In [3]:
x = np.array([1, 2, 3, 4], dtype=float)
w = np.array([3, 5, 7], dtype=float)
b = 1.0

conv = SimpleConv1d(filter_size=3)
conv.w = w.copy()
conv.b = b

# Forward
a = conv.forward(x)
print("Forward output:", a)

# Backward
delta_a = np.array([10, 20], dtype=float)
delta_x = conv.backward(delta_a)
print("Updated w:", conv.w)
print("Updated b:", conv.b)
print("Delta x:", delta_x)


Forward output: [35. 50.]
Updated w: [3. 5. 7.]
Updated b: 1.0
Delta x: None


In [4]:
class Conv1d:
    def __init__(self, input_channels, output_channels, filter_size, learning_rate=0.01):
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.filter_size = filter_size
        self.lr = learning_rate

        # Weights: (output_channels, input_channels, filter_size) float
        self.w = np.random.randn(output_channels, input_channels, filter_size).astype(float) * np.sqrt(1.0/filter_size)
        self.b = np.zeros(output_channels, dtype=float)

    def forward(self, x):
        self.x = x.astype(float)
        output_length = x.shape[1] - self.filter_size + 1
        a = np.zeros((self.output_channels, output_length), dtype=float)

        for o in range(self.output_channels):
            for i in range(self.input_channels):
                for j in range(output_length):
                    a[o, j] += np.sum(self.x[i, j:j+self.filter_size] * self.w[o, i])
            a[o] += self.b[o]

        self.a = a
        return a

    def backward(self, delta_a):
        delta_w = np.zeros_like(self.w, dtype=float)
        delta_b = np.sum(delta_a, axis=1)
        delta_x = np.zeros_like(self.x, dtype=float)

        output_length = delta_a.shape[1]

        for o in range(self.output_channels):
            for i in range(self.input_channels):
                for j in range(output_length):
                    delta_w[o, i] += delta_a[o, j] * self.x[i, j:j+self.filter_size]
                    delta_x[i, j:j+self.filter_size] += delta_a[o, j] * self.w[o, i]

        # Update weights and biases
        self.w -= self.lr * delta_w
        self.b -= self.lr * delta_b

        return delta_x


In [5]:
def pad_input(x, pad_width):
    # x: (channels, length), pad_width: int
    return np.pad(x, ((0,0),(pad_width, pad_width)), mode='constant')

x = np.array([[1,2,3,4],[2,3,4,5]], dtype=float)
x_padded = pad_input(x, 1)
print(x_padded)  # shape (2, 6)


[[0. 1. 2. 3. 4. 0.]
 [0. 2. 3. 4. 5. 0.]]


In [6]:
def pad_input(x, pad_width):
    # x: (channels, length), pad_width: int
    return np.pad(x, ((0,0),(pad_width, pad_width)), mode='constant')

x = np.array([[1,2,3,4],[2,3,4,5]], dtype=float)
x_padded = pad_input(x, 1)
print(x_padded)  # shape (2, 6)


[[0. 1. 2. 3. 4. 0.]
 [0. 2. 3. 4. 5. 0.]]


In [7]:
def forward_batch(conv_layer, batch_x):
    # batch_x: (batch_size, channels, length)
    batch_out = []
    for x in batch_x:
        batch_out.append(conv_layer.forward(x))
    return np.array(batch_out, dtype=float)


In [8]:
class Conv1dStride(Conv1d):
    def forward(self, x, stride=1):
        self.x = x.astype(float)
        output_length = (x.shape[1] - self.filter_size)//stride + 1
        a = np.zeros((self.output_channels, output_length), dtype=float)

        for o in range(self.output_channels):
            for i in range(self.input_channels):
                for j in range(output_length):
                    start = j*stride
                    a[o, j] += np.sum(x[i, start:start+self.filter_size] * self.w[o, i])
            a[o] += self.b[o]

        self.a = a
        return a


In [9]:
class Scratch1dCNNClassifier:
    def __init__(self):
        # 1 Conv1d layer + 1 fully connected layer
        self.conv = Conv1d(input_channels=1, output_channels=2, filter_size=3)
        self.fc_w = np.random.randn(2*26, 10).astype(float) * np.sqrt(1.0/26)
        self.fc_b = np.zeros(10, dtype=float)
        self.lr = 0.01

    def forward(self, x):
        a_conv = self.conv.forward(x)
        self.flatten = a_conv.flatten()
        a_fc = np.dot(self.flatten, self.fc_w) + self.fc_b
        return a_fc

    def backward(self, delta_fc):
        delta_flat = np.dot(delta_fc, self.fc_w.T).reshape(self.conv.a.shape)
        delta_x = self.conv.backward(delta_flat)

        self.fc_w -= self.lr * np.outer(self.flatten, delta_fc)
        self.fc_b -= self.lr * delta_fc
        return delta_x


In [10]:
x = np.random.rand(1,28).astype(float)
y_true = np.zeros(10, dtype=float)
y_true[3] = 1.0

cnn = Scratch1dCNNClassifier()
out = cnn.forward(x)
loss = np.sum((out - y_true)**2)
delta_fc = 2*(out - y_true)
cnn.backward(delta_fc)

print("Output:", out)
print("Loss:", loss)


Output: [-0.02813447 -0.00297937 -0.10109935 -0.11108691  0.06318653 -0.18608155
  0.09250745  0.09429938 -0.1467204  -0.04884528]
Loss: 1.3255172415579082


In [12]:
import numpy as np


class SimpleConv1d:
    def __init__(self, filter_size, learning_rate=0.01):
        self.filter_size = filter_size
        self.learning_rate = learning_rate
        self.w = np.random.randn(filter_size).astype(float) * np.sqrt(1.0 / filter_size)
        self.b = 0.0

    def forward(self, x):
        self.x = x.astype(float)
        output_length = len(self.x) - self.filter_size + 1
        self.a = np.empty(output_length, dtype=float)
        for i in range(output_length):
            self.a[i] = np.sum(self.x[i:i+self.filter_size] * self.w) + self.b
        return self.a

    def backward(self, delta_a):
        delta_w = np.zeros_like(self.w, dtype=float)
        delta_b = np.sum(delta_a)
        delta_x = np.zeros_like(self.x, dtype=float)

        for i in range(len(delta_a)):
            delta_w += delta_a[i] * self.x[i:i+self.filter_size]
            delta_x[i:i+self.filter_size] += delta_a[i] * self.w

        self.w -= self.learning_rate * delta_w
        self.b -= self.learning_rate * delta_b

        return delta_x


def conv1d_output_size(input_size, filter_size, padding=0, stride=1):
    return (input_size + 2*padding - filter_size)//stride + 1


class Conv1d:
    def __init__(self, input_channels, output_channels, filter_size, learning_rate=0.01):
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.filter_size = filter_size
        self.lr = learning_rate

        self.w = np.random.randn(output_channels, input_channels, filter_size).astype(float) * np.sqrt(1.0/filter_size)
        self.b = np.zeros(output_channels, dtype=float)

    def forward(self, x):
        self.x = x.astype(float)
        output_length = x.shape[1] - self.filter_size + 1
        a = np.zeros((self.output_channels, output_length), dtype=float)

        for o in range(self.output_channels):
            for i in range(self.input_channels):
                for j in range(output_length):
                    a[o, j] += np.sum(self.x[i, j:j+self.filter_size] * self.w[o, i])
            a[o] += self.b[o]

        self.a = a
        return a

    def backward(self, delta_a):
        delta_w = np.zeros_like(self.w, dtype=float)
        delta_b = np.sum(delta_a, axis=1)
        delta_x = np.zeros_like(self.x, dtype=float)

        output_length = delta_a.shape[1]

        for o in range(self.output_channels):
            for i in range(self.input_channels):
                for j in range(output_length):
                    delta_w[o, i] += delta_a[o, j] * self.x[i, j:j+self.filter_size]
                    delta_x[i, j:j+self.filter_size] += delta_a[o, j] * self.w[o, i]

        self.w -= self.lr * delta_w
        self.b -= self.lr * delta_b

        return delta_x


def pad_input(x, pad_width):
    return np.pad(x, ((0,0),(pad_width, pad_width)), mode='constant')


def forward_batch(conv_layer, batch_x):
    batch_out = []
    for x in batch_x:
        batch_out.append(conv_layer.forward(x))
    return np.array(batch_out, dtype=float)


class Conv1dStride(Conv1d):
    def forward(self, x, stride=1):
        self.x = x.astype(float)
        output_length = (x.shape[1] - self.filter_size)//stride + 1
        a = np.zeros((self.output_channels, output_length), dtype=float)

        for o in range(self.output_channels):
            for i in range(self.input_channels):
                for j in range(output_length):
                    start = j*stride
                    a[o, j] += np.sum(x[i, start:start+self.filter_size] * self.w[o, i])
            a[o] += self.b[o]

        self.a = a
        return a


class Scratch1dCNNClassifier:
    def __init__(self):
        self.conv = Conv1d(input_channels=1, output_channels=2, filter_size=3)
        self.fc_w = np.random.randn(2*26, 10).astype(float) * np.sqrt(1.0/26)
        self.fc_b = np.zeros(10, dtype=float)
        self.lr = 0.01

    def forward(self, x):
        a_conv = self.conv.forward(x)
        self.flatten = a_conv.flatten()
        a_fc = np.dot(self.flatten, self.fc_w) + self.fc_b
        return a_fc

    def backward(self, delta_fc):
        delta_flat = np.dot(delta_fc, self.fc_w.T).reshape(self.conv.a.shape)
        delta_x = self.conv.backward(delta_flat)

        self.fc_w -= self.lr * np.outer(self.flatten, delta_fc)
        self.fc_b -= self.lr * delta_fc
        return delta_x


x = np.random.rand(1,28).astype(float)
y_true = np.zeros(10, dtype=float)
y_true[3] = 1.0

cnn = Scratch1dCNNClassifier()
out = cnn.forward(x)
loss = np.sum((out - y_true)**2)
delta_fc = 2*(out - y_true)
cnn.backward(delta_fc)

print("Output:", out)
print("Loss:", loss)


Output: [ 0.11822238  0.40852609  0.49911199  0.53564349 -0.10747713  0.46795381
 -0.22201673 -0.59320436  0.69274832  0.20155459]
Loss: 1.797849276071204
